# GLM analysis on the main tasks
It uses nilearn and performs the following steps:
1. Load the data from fmriPrep in BIDS format
2. Iterate on the subjects to:
   1. Select the predictors and confounds for the design matrix
   2. Generate 1st level model
   3. Estimate contrast maps
3. Generate group level maps
4. Generate hMT+ mask

In [1]:
# Imports
import os
import glob
from nilearn.glm.first_level import first_level_from_bids
from nilearn.glm.first_level import make_first_level_design_matrix
from nilearn.interfaces.bids import save_glm_to_bids
from nilearn.glm import threshold_stats_img
from nilearn import plotting
from nilearn.plotting.cm import _cmap_d as nilearn_cmaps
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from nilearn.glm.second_level import SecondLevelModel
from nilearn.reporting import get_clusters_table
from nilearn.image import math_img
from nilearn.masking import apply_mask
from src import mnitools # AAL labelling

In [2]:
# Settings
data_dir = '/DATAPOOL/BRAINPLAYBACK/BIDS-BRAINPLAYBACK-TASK1/'
space_label = "MNI152NLin2009cAsym"
derivatives_folder = "derivatives/fmriprep/fmriprep"
task_label = "01"
smoothing_fwhm = 4.0
high_pass_hz = 0.007
out_dir = os.path.join(data_dir,"derivatives","nilearn_glm")

In [3]:
# sub-set of subject to process (sub-20 events still not ready)
sub_labels = ['16']
# sub_labels = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19']

## 1. Load the data from fmriPrep in BIDS format

In [4]:
# import first level data automatically from fmriPrep derivatives
(
    models,
    models_run_imgs,
    models_events,
    models_confounds,
) = first_level_from_bids(
    data_dir,
    task_label,
    space_label,
    sub_labels = sub_labels,
    hrf_model="spm",
    noise_model="ar2",
    smoothing_fwhm=smoothing_fwhm,
    high_pass=high_pass_hz,
    slice_time_ref=None,
    n_jobs=12,
    minimize_memory = False,
    derivatives_folder=derivatives_folder,
)

In [5]:
models_events[0][0]

,onset,duration,trial_type
0,0.00,11.95,Baseline
1,11.95,12.01,Noise
2,23.96,12.04,Baseline
3,36.00,11.50,Q4_p
4,36.00,11.50,Q4_db
...,...,...,...
67,540.04,11.50,Q3_db
68,552.03,11.51,Q3_p
69,552.03,11.51,Q3_db
70,564.03,11.51,Q1_p


In [5]:
# edit all models_events to exclude the conditions that end with '_db' - use the participant report (p) and not the database one (db)
for ii in range(len(models_events)):
    for jj in range(len(models_events[ii])):
        models_events[ii][jj] = models_events[ii][jj][models_events[ii][jj].trial_type.str.endswith('_db') == False]

models_events[0][2]

,onset,duration,trial_type
0,0.00,11.96,Baseline
1,11.96,12.01,Noise
2,23.97,12.02,Baseline
3,35.99,11.52,Q3_p
5,47.99,11.51,Q4_p
7,59.96,11.50,Q4_p
9,71.95,12.00,Baseline
10,83.95,12.00,Noise
11,95.95,12.05,Baseline
12,107.99,11.50,Q1_p


In [6]:
# check for the existence of all of the four conditions (Q1_p, Q2_p, Q3_p, Q4_p) in all models_events
n_of_altered_runs = 0

for ii in range(len(models_events)):
    for jj in range(len(models_events[ii])):

        trial_types = models_events[ii][jj].trial_type

        # select only the ones that end with '_p'
        trial_types = trial_types[trial_types.str.endswith('_p')]

        # find unique values
        trial_types = trial_types.unique()

        # check if all four conditions are present
        if len(trial_types) != 4:

            # add the missing condition as new row with onset 0, duration 0
            if 'Q1_p' not in trial_types:
                models_events[ii][jj] = models_events[ii][jj].append({'onset': 0, 'duration': 999, 'trial_type': 'Q1_p'}, ignore_index=True)
            if 'Q2_p' not in trial_types:
                models_events[ii][jj] = models_events[ii][jj].append({'onset': 0, 'duration': 999, 'trial_type': 'Q2_p'}, ignore_index=True)  
            if 'Q3_p' not in trial_types:
                models_events[ii][jj] = models_events[ii][jj].append({'onset': 0, 'duration': 999, 'trial_type': 'Q3_p'}, ignore_index=True)
            if 'Q4_p' not in trial_types:
                models_events[ii][jj] = models_events[ii][jj].append({'onset': 0, 'duration': 999, 'trial_type': 'Q4_p'}, ignore_index=True)

            n_of_altered_runs += 1

print('Number of altered runs: ', n_of_altered_runs)


Number of altered runs:  2


/tmp/ipykernel_21408/594841504.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  models_events[ii][jj] = models_events[ii][jj].append({'onset': 0, 'duration': 999, 'trial_type': 'Q3_p'}, ignore_index=True)
/tmp/ipykernel_21408/594841504.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  models_events[ii][jj] = models_events[ii][jj].append({'onset': 0, 'duration': 999, 'trial_type': 'Q2_p'}, ignore_index=True)
/tmp/ipykernel_21408/594841504.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  models_events[ii][jj] = models_events[ii][jj].append({'onset': 0, 'duration': 999, 'trial_type': 'Q3_p'}, ignore_index=True)


# 2. Deal with contrast names
These should be balanced here.

In [3]:
# condition names
condition_names = ['Q1_p','Q2_p','Q3_p','Q4_p']

# create strings for contrasts in the format of "condition_name - Noise"
contrasts = []

# add contrast all conditions vs. noise
contrasts.append("Q1_p + Q2_p + Q3_p + Q4_p - Noise*4")

# iterate to add the other contrasts
for condition in condition_names:
    contrasts.append(condition + " - Noise")

# add more contrasts based on the arousal and valence
contrasts.append("Q1_p*0.5 + Q2_p*0.5 - Q3_p*0.5 - Q4_p*0.5") # positive arousal vs. negative arousal
contrasts.append("Q1_p*0.5 + Q4_p*0.5 - Q3_p*0.5 - Q2_p*0.5") # positive valence vs. negative valence

# just for fun - each Q against the other
contrasts.append("Q1_p - Q2_p")
contrasts.append("Q1_p - Q3_p")
contrasts.append("Q1_p - Q4_p")
contrasts.append("Q2_p - Q3_p")
contrasts.append("Q2_p - Q4_p")
contrasts.append("Q3_p - Q4_p")

contrasts

['Q1_p + Q2_p + Q3_p + Q4_p - Noise*4',
 'Q1_p - Noise',
 'Q2_p - Noise',
 'Q3_p - Noise',
 'Q4_p - Noise',
 'Q1_p*0.5 + Q2_p*0.5 - Q3_p*0.5 - Q4_p*0.5',
 'Q1_p*0.5 + Q4_p*0.5 - Q3_p*0.5 - Q2_p*0.5',
 'Q1_p - Q2_p',
 'Q1_p - Q3_p',
 'Q1_p - Q4_p',
 'Q2_p - Q3_p',
 'Q2_p - Q4_p',
 'Q3_p - Q4_p']

In [4]:
# This is to use the contrasts as names for the output files
contrasts_renamed = ['Q1234MinusNoise',
                     'Q1MinusNoise',
                     'Q2MinusNoise',
                     'Q3MinusNoise',
                     'Q4MinusNoise',
                     'PositiveArousalMinusNegativeArousal',
                     'PositiveValenceMinusNegativeValence',
                     'Q1MinusQ2',
                     'Q1MinusQ3',
                     'Q1MinusQ4',
                     'Q2MinusQ3',
                     'Q2MinusQ4',
                     'Q3MinusQ4']

contrasts_renamed

['Q1234MinusNoise',
 'Q1MinusNoise',
 'Q2MinusNoise',
 'Q3MinusNoise',
 'Q4MinusNoise',
 'PositiveArousalMinusNegativeArousal',
 'PositiveValenceMinusNegativeValence',
 'Q1MinusQ2',
 'Q1MinusQ3',
 'Q1MinusQ4',
 'Q2MinusQ3',
 'Q2MinusQ4',
 'Q3MinusQ4']

# 3. Iterate on the subjects
This takes a while, as it estimates the multi-run GLM for each subject.

In [23]:
n_scans = 600  # number of scans
tr = 1  # repetition time is 1 second
frame_times = np.arange(n_scans) * tr  # here are the corresponding frame times

for idx in range(len(models)):

    # fetch model
    model, imgs, events, confounds = (
        models[idx],
        models_run_imgs[idx],
        models_events[idx],
        models_confounds[idx],
    )

    subject = f"sub-{model.subject_label}"

    print(f"Computing 1st level model for subject: {subject}")

    # trim confounds and replace NaNs with 0
    for ii in range(len(confounds)):
        confounds[ii] = confounds[ii][['trans_x', 'trans_x_derivative1', 'trans_x_power2', 'trans_x_derivative1_power2',
                                        'trans_y', 'trans_y_derivative1', 'trans_y_power2', 'trans_y_derivative1_power2',
                                        'trans_z', 'trans_z_derivative1', 'trans_z_power2', 'trans_z_derivative1_power2',
                                        'rot_x', 'rot_x_derivative1', 'rot_x_power2', 'rot_x_derivative1_power2',
                                        'rot_y', 'rot_y_derivative1', 'rot_y_power2', 'rot_y_derivative1_power2',
                                        'rot_z', 'rot_z_derivative1', 'rot_z_power2', 'rot_z_derivative1_power2',
                                            ]]
    
        confounds[ii] = confounds[ii].fillna(0)
    
    # create design matrices
    X_list = []
    for ii in range(len(events)):
        X = make_first_level_design_matrix(
            frame_times,
            events[ii],
            add_regs=confounds[ii],
        )

        # find the condition name that should be zeroed
        if events[ii].iloc[-1].duration == 999:
            # fetch name
            cond_name = events[ii].iloc[-1].trial_type
            # zero the condition
            X[cond_name] = 0
        
        X_list.append(X)

    # Fit and contrasts
    model.fit(imgs,design_matrices=X_list)

    # create and save z_map, t_map, and beta_map to nifti files for every contrast
    for ii in range(len(contrasts)):

        z_map = model.compute_contrast(contrasts[ii], output_type="z_score")
        t_map = model.compute_contrast(contrasts[ii], output_type="stat")
        beta_map = model.compute_contrast(contrasts[ii], output_type="effect_size")

        z_map.to_filename(os.path.join(out_dir, f"{subject}_task-{task_label}_stat-z_con-{contrasts_renamed[ii]}.nii.gz"))
        t_map.to_filename(os.path.join(out_dir, f"{subject}_task-{task_label}_stat-t_con-{contrasts_renamed[ii]}.nii.gz"))
        beta_map.to_filename(os.path.join(out_dir, f"{subject}_task-{task_label}_stat-beta_con-{contrasts_renamed[ii]}.nii.gz"))

    # Attempt to free memory
    del model, imgs, events, confounds
    

Computing 1st level model for subject: sub-12


/DATAPOOL/home/alexandresayal/anaconda3/envs/brainplayback/lib/python3.9/site-packages/nilearn/glm/first_level/first_level.py:799: UserWarning: One contrast given, assuming it for all 4 runs
  warn(f"One contrast given, assuming it for all {int(n_runs)} runs")
/DATAPOOL/home/alexandresayal/anaconda3/envs/brainplayback/lib/python3.9/site-packages/nilearn/glm/first_level/first_level.py:799: UserWarning: One contrast given, assuming it for all 4 runs
  warn(f"One contrast given, assuming it for all {int(n_runs)} runs")
/DATAPOOL/home/alexandresayal/anaconda3/envs/brainplayback/lib/python3.9/site-packages/nilearn/glm/first_level/first_level.py:799: UserWarning: One contrast given, assuming it for all 4 runs
  warn(f"One contrast given, assuming it for all {int(n_runs)} runs")
/DATAPOOL/home/alexandresayal/anaconda3/envs/brainplayback/lib/python3.9/site-packages/nilearn/glm/first_level/first_level.py:799: UserWarning: One contrast given, assuming it for all 4 runs
  warn(f"One contrast give

Computing 1st level model for subject: sub-09


/DATAPOOL/home/alexandresayal/anaconda3/envs/brainplayback/lib/python3.9/site-packages/nilearn/glm/first_level/first_level.py:799: UserWarning: One contrast given, assuming it for all 4 runs
  warn(f"One contrast given, assuming it for all {int(n_runs)} runs")
/DATAPOOL/home/alexandresayal/anaconda3/envs/brainplayback/lib/python3.9/site-packages/nilearn/glm/first_level/first_level.py:799: UserWarning: One contrast given, assuming it for all 4 runs
  warn(f"One contrast given, assuming it for all {int(n_runs)} runs")
/DATAPOOL/home/alexandresayal/anaconda3/envs/brainplayback/lib/python3.9/site-packages/nilearn/glm/first_level/first_level.py:799: UserWarning: One contrast given, assuming it for all 4 runs
  warn(f"One contrast given, assuming it for all {int(n_runs)} runs")
/DATAPOOL/home/alexandresayal/anaconda3/envs/brainplayback/lib/python3.9/site-packages/nilearn/glm/first_level/first_level.py:799: UserWarning: One contrast given, assuming it for all 4 runs
  warn(f"One contrast give